# Testing the reported peformance scores published for Electrohysterogram classification on the TPEHG dataset

In [2]:
from mlscorecheck.check.bundles.ehg import check_tpehg
from mlscorecheck.aggregated import kfolds_generator

Testing the scores reported in U. R. Acharya, V. K. Sudarshan, S. Q. Rong, Z. Tan, C. M. Lim, J. E. Koh,
S. Nayak, S. V. Bhandary, Automated detection of premature delivery using emipirical mode and wavelet packet decomposition techniques with uterine electromyogram signals, Computers in biology and medicine 85 (2017) 33–42. doi: 10.1016/j.compbiomed.2017.04.013

In [3]:
folds = list(kfolds_generator(evaluation={'dataset': {'p': 38, 'n': 262}, 'folding': {'n_folds': 5}},
                                available_scores=['acc', 'sens', 'spec']))

In [4]:
len(folds)

918

In [5]:
# the 5-fold cross-validation scores reported in the paper
scores = {'acc': 0.9447, 'sens': 0.9139, 'spec': 0.9733}
eps = 0.0001

In [6]:
results = check_tpehg(scores=scores, eps=eps, n_folds=5, n_repeats=1)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/00b3060a3747439db96cd95f7cc02e63-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/00b3060a3747439db96cd95f7cc02e63-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 73 RHS
At line 80 BOUNDS
At line 92 ENDATA
Problem MODEL has 6 rows, 11 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/an

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/7b531b08564541678a54be98b5f5a769-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/7b531b08564541678a54be98b5f5a769-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 73 RHS
At line 80 BOUNDS
At line 92 ENDATA
Problem MODEL has 6 rows, 11 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 6 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 3 rows, 6 columns (6 integer (0 of which binary)) and 12 elements
Cbc0006I The LP relaxation is infeasible or too expensive
Cuts at root node changed objective from 1.79769e+308 to -1.79769e+308
Probing was tried 0 times and created 0 c

In [7]:
results['inconsistency']

True

The results show that the scores are inconsistent with the setup, they could not be the outcome of the claimed evaluation.

## Testing the assumption of the improper use of minority oversampling

In [8]:
from mlscorecheck.aggregated import fold_partitioning_generator
from mlscorecheck.check import check_1_dataset_known_folds_mos_scores, check_1_dataset_unknown_folds_mos_scores

ImportError: cannot import name 'check_1_dataset_known_folds_mos_scores' from 'mlscorecheck.check' (/home/gykovacs/workspaces/mlscorecheck/mlscorecheck/check/__init__.py)

In [ ]:
# as reported in the paper
p_prime = 244
n = 262

In [ ]:
# first we test with the high level functionality
results = check_1_dataset_unknown_folds_mos_scores(dataset={'p': p_prime, 'n': n},
                                                    folding={'n_folds': 5},
                                                    scores=scores,
                                                    eps=eps,
                                                    verbosity=0)

In [ ]:
results['inconsistency']

False

In [ ]:
results['details'][-1]['configuration_id']

962

In [ ]:
# counting the fold configurations

count = 0
for _ in fold_partitioning_generator(p=p_prime, n=n, k=5):
    count += 1

In [ ]:
print(count)

2616607


In [ ]:
# extracting the evidence
folds = []
tptn = []

evidence = results['details'][-1]['details']

for fold in evidence['lp_configuration']['evaluations'][0]['folds']['folds']:
    folds.append((fold['fold']['p'], fold['fold']['n']))
    tptn.append((fold['fold']['tp'], fold['fold']['tn']))

In [ ]:
folds

[(1, 101), (4, 97), (40, 61), (99, 2), (100, 1)]

In [ ]:
tptn

[(1.0, 96.0), (3.0, 92.0), (38.0, 59.0), (90.0, 2.0), (96.0, 1.0)]